## Neural Network Regression Model for Car Sales Forecasting

#### Import necessary libraries and modules
We are going to import necessary libraries and modules, including MLPRegressor for neural network regression, RandomizedSearchCV for hyperparameter tuning, pandas for data manipulation, numpy for numerical operations, and StandardScaler for feature standardization. These components are typically used to build and optimize a neural network regression model for data analysis and prediction tasks.

In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV  
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

### Data Import and Processing
Let's define a function to import and process the data from a CSV file. It'll read the data, format it into a period-wise structure by combining year and month, and create a pivot table indexed by car make.

In [2]:
# Define the import_data function
def import_data():
    data = pd.read_csv(file_path)
    data['Period'] = data['Year'].astype(str) + '-' + data['Month'].astype(str).str.zfill(2)
    df = pd.pivot_table(data=data, values='Quantity', index='Make', columns='Period', aggfunc='sum', fill_value=0)
    return df

# URL of the CSV file
file_path = "https://supchains.com/wp-content/uploads/2021/07/norway_new_car_sales_by_make1.csv"

# Create the DataFrame using the import_data function
df = import_data()

# Print the DataFrame
print(df.head())

Period        2007-01  2007-02  2007-03  2007-04  2007-05  2007-06  2007-07  \
Make                                                                          
Alfa Romeo         16        9       21       20       17       21       14   
Aston Martin        0        0        1        0        4        3        3   
Audi              599      498      682      556      630      498      562   
BMW               352      335      365      360      431      477      403   
Bentley             0        0        0        0        0        1        0   

Period        2007-08  2007-09  2007-10  ...  2016-04  2016-05  2016-06  \
Make                                     ...                              
Alfa Romeo         12       15       10  ...        3        1        2   
Aston Martin        0        0        0  ...        0        0        1   
Audi              590      393      554  ...      685      540      551   
BMW               348      271      562  ...     1052      832      808

### Data Preparation
Now, we are going to define the datasets function, which will prepare the data for machine learning model training and testing. It'll take the DataFrame and specified lengths for input (x_len) and output (y_len) sequences, creating training and testing datasets. The function will arrange the data into overlapping sequences of a defined length for model training and split the data into features (X) and labels (Y). Additionally, it'll support creating test sets based on a specified number of loops, and format the data for compatibility with Scikit-learn models.

In [3]:
# Define the datasets function with x_len as an argument
def datasets(df, x_len=12, y_len=1, test_loops=12):
    D = df.values
    rows, periods = D.shape
    
    # Training set creation
    loops = periods + 1 - x_len - y_len
    train = []
    for col in range(loops):
        train.append(D[:, col:col + x_len + y_len])
    train = np.vstack(train)
    X_train, Y_train = np.split(train, [-y_len], axis=1)
    
    # Test set creation
    if test_loops > 0:
        X_train, X_test = np.split(X_train, [-rows * test_loops], axis=0)
        Y_train, Y_test = np.split(Y_train, [-rows * test_loops], axis=0)
    else:
        X_test = D[:, -x_len:]
        Y_test = np.full((X_test.shape[0], y_len), np.nan)
    
    # Formatting required for scikit-learn
    if y_len == 1:
        Y_train = Y_train.ravel()
        Y_test = Y_test.ravel()
        
    return X_train, Y_train, X_test, Y_test

### Data Preparation and Scaling
Here we are going to prepare and scale the data for the model. We'll start by splitting the dataset into training and testing sets using the datasets function. Following this, we'll employ the StandardScaler from Scikit-learn to scale the data. X_train and X_test will be transformed such that they'll have a mean of 0 and a standard deviation of 1, a standard practice to normalize data before feeding it into many machine learning algorithms. This step is crucial for models that are sensitive to the scale of the input data, such as neural networks and distance-based algorithms.

In [4]:
# Split the dataset into training and testing sets
X_train, Y_train, X_test, Y_test = datasets(df)

#Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Performance Metrics Calculation
Let's define now the kpi_ML function, which calculates key performance metrics for evaluating the  model. It computes the Mean Absolute Error (MAE), Root Mean Squared Error (RMSE), and Bias for both training and testing datasets. These metrics are essential for understanding the model's accuracy and prediction error. The results are displayed in a pandas DataFrame, providing a clear and concise summary of the model's performance.

In [5]:
# Define the kpi_ML function
def kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name=""):
    df = pd.DataFrame(columns=['MAE', 'RMSE', 'Bias'], index=['Train', 'Test'])
    df.index.name = name

    df.loc['Train', 'MAE'] = 100 * np.mean(np.abs(Y_train - Y_train_pred)) / np.mean(Y_train)
    df.loc['Train', 'RMSE'] = 100 * np.sqrt(np.mean((Y_train - Y_train_pred)**2)) / np.mean(Y_train)
    df.loc['Train', 'Bias'] = 100 * np.mean((Y_train - Y_train_pred)) / np.mean(Y_train)

    df.loc['Test', 'MAE'] = 100 * np.mean(np.abs(Y_test - Y_test_pred)) / np.mean(Y_test)
    df.loc['Test', 'RMSE'] = 100 * np.sqrt(np.mean((Y_test - Y_test_pred)**2)) / np.mean(Y_test)
    df.loc['Test', 'Bias'] = 100 * np.mean((Y_test - Y_test_pred)) / np.mean(Y_test)

    df = df.astype(float).round(1)  # Round numbers for display
    print(df)

### Configuring and Training a Neural Network Regressor
We are going to set up and train the neural network using Scikit-learn's MLPRegressor. We’ll begin by defining a set of fixed parameters (param_fixed), including the activation function ('relu'), optimizer ('adam'), and settings for early stopping, validation fraction, tolerance, and maximum iterations. Then, an MLPRegressor model will be instantiated with these parameters and a predefined hidden layer size. The model is then trained using scaled training data (X_train_scaled, Y_train) and subsequently makes predictions on both training and testing data. Finally, the function kpi_ML is called to evaluate the model's performance.

In [6]:
param_fixed = {
    'activation': 'relu',
    'solver': 'adam',
    'early_stopping': True,
    'n_iter_no_change': 30,
    'validation_fraction': 0.1,
    'tol': 0.0001,
    'max_iter': 2000  
}

NN = MLPRegressor(hidden_layer_sizes=(20, 20), **param_fixed, verbose=False)
NN.fit(X_train_scaled, Y_train)
Y_train_pred = NN.predict(X_train_scaled) 
Y_test_pred = NN.predict(X_test_scaled) 
kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name='NN')  

        MAE  RMSE  Bias
NN                     
Train  17.5  44.0  -0.5
Test   17.6  43.6   1.0


### Optimized Hyperparameter Grid for MLPRegressor Tuning
Here we define an enhanced range of hyperparameters for tuning the MLP. It includes varied hidden_layer_sizes, logarithmically scaled alpha and learning_rate_init values, and refined ranges for beta_1 and beta_2. This setup aims to improve the efficiency and effectiveness of the MLP tuning process.

In [7]:
hidden_layer_sizes = [(x, y) for x in range(10, 60, 10) for y in range(10, 60, 10)]
alpha = np.logspace(-3, 1, 5)
learning_rate_init = np.logspace(-5, -1, 5)
beta_1 = np.linspace(0.85, 0.99, 5)
beta_2 = np.linspace(0.99, 0.9999, 5)
param_dist = {
    'hidden_layer_sizes': hidden_layer_sizes, 
    'alpha': alpha, 
    'learning_rate_init': learning_rate_init, 
    'beta_1': beta_1, 
    'beta_2': beta_2
}
NN = MLPRegressor(**param_fixed)
NN_cv = RandomizedSearchCV(NN, param_dist, cv=10, verbose=0, n_jobs=-1, n_iter=200, scoring='neg_mean_absolute_error')
NN_cv.fit(X_train_scaled, Y_train)

# Extract the best parameters from the randomized search
best_params = NN_cv.best_params_

# Create a new MLPRegressor model with the best parameters
optimized_NN = MLPRegressor(**best_params)

# Fit the model with your scaled training data
optimized_NN.fit(X_train_scaled, Y_train)

# Predict using the optimized model
Y_train_pred = optimized_NN.predict(X_train_scaled)
Y_test_pred = optimized_NN.predict(X_test_scaled)

# Evaluate the performance of the optimized model
kpi_ML(Y_train, Y_train_pred, Y_test, Y_test_pred, name='NN optimized')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


               MAE  RMSE  Bias
NN optimized                  
Train         17.4  43.8  -0.2
Test          17.6  43.6   1.3


### Generating forecasts using the optimized model


In [8]:
# Splitting and scaling the data
X_train, Y_train, X_test, Y_test = datasets(df, x_len=12, y_len=1, test_loops=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Setting up and fitting the randomized search
NN = MLPRegressor(**param_fixed)
NN_cv = RandomizedSearchCV(NN, param_dist, cv=10, verbose=0, n_jobs=-1, n_iter=200, scoring='neg_mean_absolute_error')
NN_cv.fit(X_train_scaled, Y_train)  

best_params = NN_cv.best_params_
optimized_NN = MLPRegressor(**best_params)
optimized_NN.fit(X_train_scaled, Y_train)  # Fit the optimized model

# Generating forecasts using the optimized model
forecast = pd.DataFrame(data=optimized_NN.predict(X_test_scaled), index=df.index)
print(forecast)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


                        0
Make                     
Alfa Romeo       6.221273
Aston Martin     1.034259
Audi           649.657690
BMW           1256.049390
Bentley          1.165328
...                   ...
Think            1.098836
Toyota        1450.411142
Volkswagen    1994.160108
Volvo          923.392109
Westfield        1.098836

[65 rows x 1 columns]
